## 1. Objective:
* Primary Objective : To develop a real-time emotion recognition system that can detect and classify human facial expressions as "angry," "happy," "sad," "fear," "disgust," "surprise," and "neutral" from a live webcam feed.
* Use Case: The system is designed to be deployed for various applications, such as human-computer interaction, behavioral analysis, customer feedback systems, and interactive gaming.

## 2. Goal:
* To accurately predict emotions from facial expressions captured by a webcam in real-time.
* To implement face detection using YOLOv8 for localizing faces and emotion classification using a CNN model.
* To create a system that can adapt to varying real-time conditions such as lighting changes, different facial angles, and diverse expressions.
* To deliver a user-friendly interface that displays the predicted emotion with high accuracy.

In [1]:
# !pip install tensorflow keras numpy==1.24.4 opencv-python==4.5.5.64 opencv-python-headless==4.5.5.64 ultralytics matplotlib

# Step 1: Import Libraries

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Step 2: Define Dataset Path
Set up the path to your dataset in your local directory.

In [3]:
base_path = r"D:\Data Science 6pm\Assessment\Facial_detection"
train_dir = os.path.join(base_path, "train")
test_dir = os.path.join(base_path, "test")

# Step 3: Preprocess Dataset
Use ImageDataGenerator to preprocess the dataset for training and testing.

In [4]:
datagen = ImageDataGenerator(rescale=1./255)  # Normalize images to [0, 1]

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # Resize images to 48x48
    batch_size=32,
    class_mode='categorical'
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

Found 28778 images belonging to 7 classes.
Found 7248 images belonging to 7 classes.


# Step 4: Build and Train the Emotion Detection Model
Build a Convolutional Neural Network (CNN) to classify the seven emotions.

In [5]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for emotions
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=80
)




Epoch 1/80


900/900 [==============================] - 61s 66ms/step - loss: 1.6847 - accuracy: 0.3299 - val_loss: 1.4993 - val_accuracy: 0.4262
Epoch 2/80
900/900 [==============================] - 37s 41ms/step - loss: 1.5087 - accuracy: 0.4154 - val_loss: 1.4109 - val_accuracy: 0.4587
Epoch 3/80
900/900 [==============================] - 35s 39ms/step - loss: 1.4324 - accuracy: 0.4470 - val_loss: 1.3666 - val_accuracy: 0.4765
Epoch 4/80
900/900 [==============================] - 35s 38ms/step - loss: 1.3671 - accuracy: 0.4701 - val_loss: 1.3358 - val_accuracy: 0.4898
Epoch 5/80
900/900 [==============================] - 59s 66ms/step - loss: 1.3215 - accuracy: 0.4910 - val_loss: 1.2778 - val_accuracy: 0.5054
Epoch 6/80
900/900 [==============================] - 55s 61ms/step - loss: 1.2759 - accuracy: 0.5076 - val_loss: 1.2772 - val_accuracy: 0.5047
Epoch 7/80
900/900 [==============================] - 41s 46ms/step - loss: 1.2341 - accuracy: 0.5240 - val_loss: 1.2525 - val_accu

Epoch 50/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4922 - accuracy: 0.7910 - val_loss: 2.6004 - val_accuracy: 0.5440
Epoch 51/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4912 - accuracy: 0.7905 - val_loss: 2.5195 - val_accuracy: 0.5403
Epoch 52/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4881 - accuracy: 0.7899 - val_loss: 2.5032 - val_accuracy: 0.5384
Epoch 53/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4770 - accuracy: 0.7954 - val_loss: 2.4912 - val_accuracy: 0.5454
Epoch 54/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4805 - accuracy: 0.7969 - val_loss: 2.7423 - val_accuracy: 0.5450
Epoch 55/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4742 - accuracy: 0.8017 - val_loss: 2.5728 - val_accuracy: 0.5401
Epoch 56/80
900/900 [==============================] - 36s 40ms/step - loss: 0.4749 - accuracy: 0.7974 - val_loss: 2.5876 - val_ac

## Save the trained model:

In [6]:
model.save("emotion_classifier.h5")

C:\Users\SAMEER\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Step 5: Set Up YOLO for Face Detection
Download the YOLO model (yolov8n.pt) and load it. Ensure the file is in the working directory or specify its full path.

In [7]:
# Load YOLO model for face detection
yolo = YOLO("yolov8n.pt")  # Ensure yolov8n.pt is in your working directory

# Step 6: Real-Time Emotion Detection
Use OpenCV for webcam feed and YOLO for face detection, combined with your trained emotion model.

In [8]:
# Load trained emotion classifier
emotion_model = load_model('emotion_classifier.h5')
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def preprocess_face(face_img):
    """Resize and normalize face image for emotion classification."""
    face_img = cv2.resize(face_img, (48, 48))  # Resize to model input size
    face_img = face_img / 255.0  # Normalize pixel values
    face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
    return face_img

# Start webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

print("Press 'q' to quit the application.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # YOLO face detection
    results = yolo.predict(frame, conf=0.5)
    detections = results[0].boxes.xyxy.numpy()  # Bounding boxes

    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])  # Extract bounding box coordinates
        face = frame[y1:y2, x1:x2]  # Crop the detected face

        if face.size > 0:
            # Preprocess face and predict emotion
            preprocessed_face = preprocess_face(face)
            emotion_prediction = emotion_model.predict(preprocessed_face, verbose=0)
            emotion = emotion_labels[np.argmax(emotion_prediction)]  # Get emotion label

            # Draw bounding box and emotion label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow('Real-Time Emotion Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Press 'q' to quit the application.

0: 480x640 1 person, 262.1ms
Speed: 39.0ms preprocess, 262.1ms inference, 16.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 126.5ms
Speed: 4.0ms preprocess, 126.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 112.1ms
Speed: 4.0ms preprocess, 112.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 143.5ms
Speed: 5.5ms preprocess, 143.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 109.5ms
Speed: 3.0ms preprocess, 109.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 120.5ms
Speed: 3.0ms preprocess, 120.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 109.5ms
Speed: 3.0ms preprocess, 109.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 187.1ms
Speed


0: 480x640 1 person, 1 refrigerator, 101.6ms
Speed: 2.0ms preprocess, 101.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 123.0ms
Speed: 2.0ms preprocess, 123.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 118.0ms
Speed: 3.0ms preprocess, 118.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 121.0ms
Speed: 2.0ms preprocess, 121.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 118.1ms
Speed: 3.0ms preprocess, 118.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 109.5ms
Speed: 2.0ms preprocess, 109.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 114.6ms
Speed: 2.0ms preprocess, 114.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x


0: 480x640 1 person, 92.5ms
Speed: 3.0ms preprocess, 92.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 92.5ms
Speed: 2.1ms preprocess, 92.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 89.6ms
Speed: 1.0ms preprocess, 89.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 94.6ms
Speed: 3.0ms preprocess, 94.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 110.6ms
Speed: 2.0ms preprocess, 110.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 153.5ms
Speed: 11.0ms preprocess, 153.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 122.5ms
Speed: 3.0ms preprocess, 122.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 123.5ms
Speed: 3.0ms preprocess, 123.5ms 


0: 480x640 1 person, 91.6ms
Speed: 2.0ms preprocess, 91.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 101.5ms
Speed: 2.0ms preprocess, 101.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 92.6ms
Speed: 2.0ms preprocess, 92.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 91.5ms
Speed: 1.0ms preprocess, 91.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 90.5ms
Speed: 2.0ms preprocess, 90.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 97.5ms
Speed: 2.0ms preprocess, 97.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 89.5ms
Speed: 2.0ms preprocess, 89.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 90.5ms
Speed: 2.0ms preprocess, 90.5ms inference, 1.0ms postprocess per image at shape (1, 3, 


0: 480x640 1 person, 91.5ms
Speed: 2.0ms preprocess, 91.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 91.0ms
Speed: 2.5ms preprocess, 91.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 90.5ms
Speed: 2.0ms preprocess, 90.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 91.7ms
Speed: 2.0ms preprocess, 91.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 89.5ms
Speed: 1.0ms preprocess, 89.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 97.5ms
Speed: 1.0ms preprocess, 97.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 95.5ms
Speed: 2.0ms preprocess, 95.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 89.5ms
Speed: 2.0ms preprocess, 89.5ms inference, 1.0ms postprocess per image at shape (1, 3, 48


0: 480x640 1 person, 102.5ms
Speed: 2.0ms preprocess, 102.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 93.5ms
Speed: 2.0ms preprocess, 93.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 100.5ms
Speed: 2.0ms preprocess, 100.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 104.6ms
Speed: 3.0ms preprocess, 104.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 94.5ms
Speed: 2.0ms preprocess, 94.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 103.5ms
Speed: 2.0ms preprocess, 103.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 102.6ms
Speed: 2.0ms preprocess, 102.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 101.5ms
Speed: 1.5ms preprocess, 101.5ms inference, 1.0ms postprocess per image at sha


0: 480x640 1 person, 115.5ms
Speed: 2.0ms preprocess, 115.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 115.5ms
Speed: 3.0ms preprocess, 115.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 123.5ms
Speed: 3.0ms preprocess, 123.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 114.5ms
Speed: 3.0ms preprocess, 114.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 110.6ms
Speed: 3.0ms preprocess, 110.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 114.1ms
Speed: 2.0ms preprocess, 114.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 127.1ms
Speed: 3.0ms preprocess, 127.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 117.5ms
Speed: 2.0ms preprocess, 117.5ms inference, 1.0ms postprocess per image at


0: 480x640 1 person, 111.1ms
Speed: 2.0ms preprocess, 111.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 94.0ms
Speed: 2.6ms preprocess, 94.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 110.5ms
Speed: 2.0ms preprocess, 110.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 98.5ms
Speed: 2.0ms preprocess, 98.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 102.5ms
Speed: 1.0ms preprocess, 102.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 99.5ms
Speed: 2.0ms preprocess, 99.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 101.5ms
Speed: 2.0ms preprocess, 101.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 109.5ms
Speed: 2.0ms preprocess, 109.5ms inference, 1.0ms postprocess per image at shape


0: 480x640 1 person, 118.5ms
Speed: 3.0ms preprocess, 118.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 104.5ms
Speed: 1.0ms preprocess, 104.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 refrigerator, 99.5ms
Speed: 2.0ms preprocess, 99.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 97.6ms
Speed: 1.0ms preprocess, 97.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 104.6ms
Speed: 2.0ms preprocess, 104.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 96.6ms
Speed: 1.0ms preprocess, 96.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 103.5ms
Speed: 2.0ms preprocess, 103.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


# Summary
* Train the model using your dataset in D: drive
* Save the trained model (emotion_classifier.h5) for later use.
* Use YOLO for face detection and the trained model for real-time emotion prediction. 

## Description:
* Face Detection: The project begins by using the YOLOv8 model for detecting faces in the real-time webcam feed. YOLO (You Only Look Once) is a highly efficient deep learning model designed for object detection, ensuring fast and reliable face localization.
* Emotion Classification: Once a face is detected, a Convolutional Neural Network (CNN) is used to classify the facial expression into one of the predefined emotional classes: angry, disgust, fear, happy, neutral, sad, and surprise. The CNN is trained on a labeled dataset containing facial images of people showing different emotions.
* Real-Time Processing: The webcam captures frames, and for each frame, the system detects faces and predicts emotions continuously. The predicted emotion is displayed on the screen, making the system suitable for interactive use.
* Technology Stack:
1. YOLOv8 for real-time face detection.
2. Keras/TensorFlow for training and implementing the emotion classification model.
3. OpenCV for webcam interaction, image pre-processing, and displaying results.

## Result:
* The system successfully detects faces from a live webcam feed and classifies the facial expressions with good accuracy.
* Model Performance: The emotion classifier provides reliable predictions, with higher confidence for expressions like "happy" and "angry," though performance may vary under different conditions like poor lighting or occlusion.
* User Interaction: The interface updates in real-time with the emotion prediction displayed on the screen as soon as the user changes their expression in front of the webcam. 